# 自动生成金庸文

In [17]:
import os
import urllib
import zipfile
import codecs

with zipfile.ZipFile("./sd.zip", "r") as f:
    f.extractall("./")     
    
with codecs.open('sd.txt', encoding='utf-8') as f:
    print f.read()[0:1000]



射雕英雄传

金庸

第一回风雪惊变

钱塘江浩浩江水，日日夜夜无穷无尽的从临安牛家村边绕过，东流入海
江畔一排数十株乌柏树，叶子似火烧般红，正是八月天时
村前村后的野草刚起始变黄，一抹斜阳映照之下，更增了几分萧索
两株大松树下围着一堆村民，男男女女和十几个小，正自聚精会神的听着一个瘦削的老者说话

那说话人五十来岁年纪，一件青布长袍早洗得褪成了蓝灰色
只听他两片梨花木板碰了几下，右手中竹棒在一面小羯鼓上敲起得得连声
唱道：小桃无主自开花，烟草茫茫带晚鸦

几处败垣围故井，向来一一是人家

那说话人将木板敲了几下，说道：这首七言诗，说的是兵火过后，原来是家家户户，都变成了断墙残瓦的破败之地
小人刚才说到那叶老汉一家四口，悲欢离合，聚了又散，散了又聚
他四人给金兵冲散，好容易又再团聚，欢天喜地的回到故乡，却见房屋已给金兵烧得干干净净，无可奈何，只得去到汴梁，想觅个生计
不料想：天有不测风云，人有旦夕祸福
他四人刚进汴梁城，迎面便过来一队金兵
带兵的头儿一双三角眼觑将过去，只见那叶三姐生得美貌，跳下马来，当即一把抱住，哈哈大笑，便将她放上了马鞍，说道：‘小姑娘，跟我回家，服侍老爷
’那叶三姐如何肯从
拼命挣扎
那金兵长官喝道：‘你不肯从我，便杀了你的父母兄弟／提起狼牙棒，一棒打在叶四郎的头上，登时脑浆迸裂，一命呜呼
正是：阴间新添枉死鬼，阳间不见少年人１叶老汉和妈妈吓得呆了，扑将上去，搂住了儿子的死尸，放声大哭
那长官提起狼牙棒，一棒一个，又都了账
那叶三姐却不啼哭，说道：‘长官休得凶恶，我跟你回家便了．那长官大喜，将叶三姐带得回家
不料叶三姐觑他不防，突然抢步过去，拔出那长官的腰刀，对准了他心口，一刀刺将过去，说时迟，那时快，这一刀刺去，眼见便可报得父母兄弟的大仇
不料那长官久经战阵，武艺精熟，顺手一推，叶三姐登时摔了出去，那长官刚骂得一声：‘小贱人／叶三姐已举起钢刀，在脖子中一勒
可怜她：花容月貌无双女，惆怅芳魂赴九泉

他说一段，唱一段，只听得众村民无不咬牙切齿，愤怒叹息

那人又说道：众位看官，常言道得好：为人切莫用欺心，举头三尺有神明

若还作恶无报应，天下凶徒人吃人

可是那金兵占了我大宋天下，杀人放火，奸淫掳掠，无恶不作，却又不见他遭到什么报应
只怪我大宋官家不争气，


In [18]:
def read_content(path):
    with codecs.open(path, encoding='utf-8') as ins:
        return ins.read()
    
# Return a dict which maps each char into an unique int id
def build_vocab(path):
    content = list(read_content(path))
    idx = 1 # 0 is left for zero-padding
    the_vocab = {}
    for word in content:
        if len(word) == 0:
            continue
        if not word in the_vocab:
            the_vocab[word] = idx
            idx += 1
    return the_vocab

# Encode a sentence with int ids
def text2id(sentence, the_vocab):
    words = list(sentence)
    return [the_vocab[w] for w in words if len(w) > 0]
            
# build char vocabluary from input
vocab = build_vocab("./sd.txt")
print('vocab size = %d' %(len(vocab)))

vocab size = 4442


## Create LSTM Model

In [19]:
import lstm
# Each line contains at most 129 chars. 
seq_len = 151
# embedding dimension, which maps a character to a 256-dimension vector
num_embed = 256
# number of lstm layers
num_lstm_layer = 3
# hidden unit in LSTM cell
num_hidden = 512

symbol = lstm.lstm_unroll(
    num_lstm_layer, 
    seq_len,
    len(vocab) + 1,
    num_hidden=num_hidden,
    num_embed=num_embed,
    num_label=len(vocab) + 1, 
    dropout=0.2)

"""test_seq_len"""
data_file = codecs.open("./sd.txt", encoding='utf-8')
for line in data_file:
    assert len(line) <= seq_len + 1, "seq_len is smaller than maximum line length. Current line length is %d. Line content is: %s" % (len(line), line)
data_file.close()

## Train Model

In [20]:
import bucket_io

# The batch size for training
batch_size = 32

# initalize states for LSTM
init_c = [('l%d_init_c'%l, (batch_size, num_hidden)) for l in range(num_lstm_layer)]
init_h = [('l%d_init_h'%l, (batch_size, num_hidden)) for l in range(num_lstm_layer)]
init_states = init_c + init_h

# Even though BucketSentenceIter supports various length examples,
# we simply use the fixed length version here
data_train = bucket_io.BucketSentenceIter(
    "./sd.txt", 
    vocab, 
    [seq_len], 
    batch_size,             
    init_states, 
    seperate_char='\n',
    text2id=text2id, 
    read_content=read_content)

Summary of dataset ==================
bucket of len 151 : 30463 samples


In [ ]:
# @@@ AUTOTEST_OUTPUT_IGNORED_CELL
import mxnet as mx
import numpy as np
import logging
logging.getLogger().setLevel(logging.DEBUG)

# We will show a quick demo with only 1 epoch. In practice, we can set it to be 100
num_epoch = 1
# learning rate 
learning_rate = 0.01

# Evaluation metric
def Perplexity(label, pred):
    loss = 0.
    for i in range(pred.shape[0]):
        loss += -np.log(max(1e-10, pred[i][int(label[i])]))
    return np.exp(loss / label.size)

model = mx.model.FeedForward(
    ctx=mx.gpu(0),
    symbol=symbol,
    num_epoch=num_epoch,
    learning_rate=learning_rate,
    momentum=0,
    wd=0.0001,
    initializer=mx.init.Xavier(factor_type="in", magnitude=2.34))

model.fit(X=data_train,
          eval_metric=mx.metric.np(Perplexity),
          batch_end_callback=mx.callback.Speedometer(batch_size, 20),
          epoch_end_callback=mx.callback.do_checkpoint("sdtxt"))

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:26: DeprecationWarning: mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.
INFO:root:Start training with [gpu(0)]


## Inference

In [ ]:
from rnn_model import LSTMInferenceModel


# helper strcuture for prediction
def MakeRevertVocab(vocab):
    dic = {}
    for k, v in vocab.items():
        dic[v] = k
    return dic

# make input from char
def MakeInput(char, vocab, arr):
    idx = vocab[char]
    tmp = np.zeros((1,))
    tmp[0] = idx
    arr[:] = tmp

# helper function for random sample 
def _cdf(weights):
    total = sum(weights)
    result = []
    cumsum = 0
    for w in weights:
        cumsum += w
        result.append(cumsum / total)
    return result

def _choice(population, weights):
    assert len(population) == len(weights)
    cdf_vals = _cdf(weights)
    x = random.random()
    idx = bisect.bisect(cdf_vals, x)
    return population[idx]

# we can use random output or fixed output by choosing largest probability
def MakeOutput(prob, vocab, sample=False, temperature=1.):
    if sample == False:
        idx = np.argmax(prob, axis=1)[0]
    else:
        fix_dict = [""] + [vocab[i] for i in range(1, len(vocab) + 1)]
        scale_prob = np.clip(prob, 1e-6, 1 - 1e-6)
        rescale = np.exp(np.log(scale_prob) / temperature)
        rescale[:] /= rescale.sum()
        return _choice(fix_dict, rescale[0, :])
    try:
        char = vocab[idx]
    except:
        char = ''
    return char